# CHAPTER2 Nuxt.jsによるシンプルなアプリケーション開発

## VueCLIによるアプリケーションひな形の作成

* VueCLIのインストール
```bash
npm install @vue/cli @vue/cli-init
```
* VueCLIによるプロジェクト作成
    * 基本コマンド
    ```bash
    vue init <template-name> <project-name>
    ```
    * Nuxtテンプレートの利用
    ```bash
    vue init nuxt-community/starter-template <project-name>
    ```


## 2.9 Nuxt.jsプロジェクトの構成について

* assets
    * 画像リソース、CSSなどの静的ファイルを配置するディレクトリ
    * JavaScript以外のソースファイルを配置する

* components
  * Vue.jsのコンポーネントを配置するディレクトリ
  * Nuxt.jsにおいても通常のVue.jsと同等の扱い

* pages
  * ページコンポーネントを配置するディレクトリ
  * ルーティングに対応するコンポーネントを配置する

* static
  * webpackなどを介さずに、そのまま公開するファイルを配置するディレクトリ
  * ファイル名そのままにroot上に配置する。ファイル名が決まっているものなどを置く。

* store
  * Vuexストアとそのモジュールを管理する
  * ここに配置されているモジュールとストアは、自動的にVuexストアに登録される

## 2.10 ルーティングとページコンポーネントの作成

動的ルーティングは、ファイル名の接頭辞に`_`を付けることで可能
* 動的ルーティングとは、`users/potato04`などのように、パスの一部に変数を含めることができるルーティングのこと
  * このようにすることで、Vue側からでも、`$route.params`でパラメータを取得でき、個別ユーザーごとの対応が可能

* pages/users/index.vue 
* pages/users/_id.vue : 動的ルーティングは、ファイル名の接頭辞に`_`を付ける

Nuxt.jsは`/, _`から始まるファイルを動的ルーティングとしてあつかう。
* ただし、`users/registar`という静的ルーティング（静的ページコンポーネント）がある場合`users/_id.vue`などの動的ページコンポーネントよりも優先的に表示される。
  * 静的コンポーネント　＞　動的コンポーネント
  

## 2.11 動的ルーティングのコンテンツのだし分け

動的コンポーネントのパラメータは、`this.$route.params.<id>`で受け取り可能
```vue
<template>
    <div>
        <p>
            UserID: {{ userId }}
        </p>
    </div>
</template>

<script>
    export default {
        data() {
            return {
                userId: this.$route.params.id
            }
        }
    }
</script>
```


## 2.12 axios-moduleによる外部リソースの取得

Nuxtにおいて中心となる、HTTPリクエスト用のライブラリ。

```bash
npm install @nuxt/axios
```
インストール後は、`nuxt.config.js`の`module, axios`プロパティを変更する。
```js
//nuxt.config.js

{
    ...
    modules: [
        '@nuxt/axios'
    ],
    axios: {
        ...
    }
}
```

## 2.13 承認情報を付与したAPIリクエスト

`~/plugins/axios.js`の中に、トークンを付与する処理を記述する。
```js
// ~/plugins/axios.js
export default {
    $axios.onReauest((config) =>{
        if(process.ecn.QIITA_TOKEN) {
            config.headers.common['Authorization'] = process.env.QIITA_TOKEN;
        }
        return config;
    });
}
```
設定したトークンは、`nuxt.config.js`の`env`プロパティで設定する。
```js
{
    ...
    env: {
        QIITA_TOKEN: process.env.QIITA_TOKEN
    }
}
```
Nuxt.js自体はマシンの環境変数にアクセスすることはできないのでNode.jsを介して行う。


## 2.14 非同期通信を含むコンテンツのSSR

Nuxt.jsでは基本的に、asyncDataメソッドを利用して、非同期通信を行い、コンポーネントの構築を行う。
```js
export default {
    asyncData({ $axios }) {
        return $axios.$get('https://qiita.com/api/v2/users/potato04')
            .then((res) => {
                return {
                    user: res
                }
            })
            .catch((e) => {
                return {
                    user: null
                }
            })
    }
}
```
なお、asyncDataに入る情報は、総称として、`context`と呼ばれるオブジェクトに格納されている。

## 2.16 headによるメタ情報の設定

Nuxt.jsでは、`head`プロパティを利用して、ページのメタ情報を設定することができる。
```js
export default {
    head: {
        title: 'Nuxt.jsのサンプル',
        meta: [
            { charset: 'utf-8' },
            { name: 'viewport', content: 'width=device-width, initial-scale=1' },
            { hid: 'description', name: 'description', content: 'Nuxt.jsのサンプルです。' }
        ]
    }
}
```
要するに、`<head>`タグの中身を設定することができる。

## 2.17 ロジックのVuexストアへの移譲

Vuexストアは、`store`ディレクトリにjsファイルを配置する。
```js
// store/index.js
export default () => {
    new Vuex.Store({
        state: {
            items: []
        }
    });
}
```
